In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as io:

    SETTING = yaml.load(io)

PATH = make_path_dict(SETTING)

In [ ]:
if SETTING["signal_type"] != "raw":

    raise

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [ ]:
xaxis = {"title": "Rank"}

yaxis = {"title": "Raw Signal Standard Deviation"}

In [ ]:
if SETTING["select_feature_automatically"]:

    features = kraft.select_series_indices(
        feature_x_sample_processed.std(axis=1),
        ">",
        standard_deviation=SETTING["select_feature_standard_deviation"],
        title={"text": feature_x_sample_processed.index.name},
        xaxis=xaxis,
        yaxis=yaxis,
    )

else:

    features = feature_x_sample_processed.index

f"Selected {features.size} feature."

In [ ]:
if SETTING["select_sample_automatically"]:

    samples = kraft.select_series_indices(
        feature_x_sample_processed.std(),
        ">",
        standard_deviation=SETTING["select_sample_standard_deviation"],
        title={"text": feature_x_sample_processed.columns.name},
        xaxis=xaxis,
        yaxis=yaxis,
    )

else:

    samples = feature_x_sample_processed.columns

f"Selected {samples.size} sample."

In [ ]:
raw_signal = feature_x_sample_processed.loc[features, samples]

In [ ]:
if SETTING["signal_normalization_method"] is not None:

    raw_signal = kraft.normalize_series_or_dataframe(
        raw_signal,
        SETTING["signal_normalization_axis"],
        SETTING["signal_normalization_method"],
    )

In [ ]:
raw_signal.to_csv(PATH["feature_x_sample.signal.tsv"], sep="\t")

kraft.summarize_feature_x_sample(
    raw_signal,
    feature_x_sample_alias=SETTING["feature_x_sample_alias"],
    feature_x_sample_value_name="Signal",
    plot_heat_map_max_size=SETTING["plot_heat_map_max_size"],
    plot_histogram_max_size=SETTING["plot_histogram_max_size"],
    plot_rug_max_size=SETTING["plot_rug_max_size"],
)